In [1]:
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras import backend as K
from keras.models import load_model

from keras_ssd7 import build_model
from keras_ssd_loss import SSD_Loss
from ssd_box_encode_decode_utils import SSDBoxEncoder, decode_y
from ssd_batch_generator import parse_CSV, generate_batch

Using TensorFlow backend.


In [2]:
### Set up the model

# 1: Set some necessary parameters

img_height = 300 # Height of the input images
img_width = 480 # Width of the input images
img_channels = 3 # Number of color channels of the input images
n_classes = 2 # Number of classes including the background class
min_scale=0.1 # The scaling factor for the smallest anchor boxes
max_scale=0.8 # The scaling factor for the largest anchor boxes
scales=[0.08, 0.16, 0.32, 0.64] # An explicit list of anchor box scaling factors. If this is passed, it will override `min_scale` and `max_scale`.
aspect_ratios=[0.5, 1, 2] # The list of aspect ratios for the anchor boxes
two_boxes_for_ar1=True # Whether or not you want to generate two anchor boxes for aspect ratio 1
limit_boxes=True # Whether or not you want to limit the boxes to lie entirely within the image boundaries

# 2: Build the Keras model (and possibly load some trained weights)

K.clear_session() # Clear previous models from memory.
# The output `classifier_sizes` is needed below to set up `SSDBoxEncoder`
model, classifier_sizes = build_model(image_size=(img_height, img_width, img_channels),
                                      n_classes=n_classes,
                                      min_scale=min_scale,
                                      max_scale=max_scale,
                                      scales=scales,
                                      aspect_ratios=aspect_ratios,
                                      two_boxes_for_ar1=two_boxes_for_ar1,
                                      limit_boxes=limit_boxes)
#model.load_weights('./model_0_weights.h5')
#model = load_model('./model_0.h5')

In [3]:
### Set up training

batch_size = 32

# 3: Instantiate an Adam optimizer and the SSD loss function and compile the model

adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=5e-05)

ssd_loss = SSD_Loss(neg_pos_ratio=3, n_neg_min=0, alpha=1.0)

model.compile(optimizer=adam, loss=ssd_loss.compute_loss)

# 4: Instantiate an encoder that can encode ground truth labels into the format needed by the SSD loss function 

ssd_box_encoder = SSDBoxEncoder(img_height=img_height,
                                img_width=img_width,
                                n_classes=n_classes,
                                classifier_sizes=classifier_sizes,
                                min_scale=min_scale,
                                max_scale=max_scale,
                                scales=scales,
                                aspect_ratios=aspect_ratios,
                                two_boxes_for_ar1=two_boxes_for_ar1,
                                limit_boxes=limit_boxes,
                                pos_iou_threshold=0.5,
                                neg_iou_threshold=0.3,
                                coords='centroids')

# 5: Create the batch generator

filenames, labels = parse_CSV('./data/labels.csv', n_classes)

train_generator = generate_batch(filenames=filenames,
                                 labels=labels,
                                 path_prefix='./data/',
                                 batch_size=batch_size,
                                 train=True,
                                 ssd_box_encoder=ssd_box_encoder,
                                 crop=False,
                                 resize=False,
                                 gray=False,
                                 equalize=False,
                                 brightness=(0.5, 2, 0.7),
                                 flip=0.5,
                                 translate=((10, 200), (10, 150), 0.7),
                                 scale=(0.75, 1.3, 0.7),
                                 limit_boxes=True,
                                 include_thresh=0.4,
                                 diagnostics=False)

In [ ]:
### Run training

# 6: Run training

epochs = 10

history = model.fit_generator(generator = train_generator,
                              steps_per_epoch = int(len(filenames)/batch_size)+1,
                              epochs = epochs,
                              callbacks = [ModelCheckpoint('./model_0_weights_{epoch:02d}_{loss:.2f}.h5',
                                                           monitor='loss',
                                                           verbose=1,
                                                           save_best_only=True,
                                                           save_weights_only=True,
                                                           mode='auto',
                                                           period=1),
                                           EarlyStopping(monitor='loss',
                                                         min_delta=0.1,
                                                         patience=5),
                                           ReduceLROnPlateau(monitor='loss',
                                                             factor=0.5,
                                                             patience=3,
                                                             epsilon=0.1,
                                                             cooldown=0)])

model_name = 'model_0'
model.save('./{}.h5'.format(model_name))
model.save_weights('./{}_weights.h5'.format(model_name))

print()
print("Model saved as {}.h5".format(model_name))
print("Weights also saved separately as {}_weights.h5".format(model_name))
print()